# Analyzing a large dataset on the IP2I SLURM Farm

## The New York City taxi dataset

In [6]:
import pprint
import pathlib

taxis = pathlib.Path("/gridgroup/cms/cbernet/test_data/nyc_taxis")

t2009 = sorted(list((taxis / "2009").glob("*.csv")))
pprint.pprint(t2009)

[PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-01.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-02.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-03.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-04.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-05.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-06.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-07.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-08.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-09.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-10.csv'),
 PosixPath('/gridgroup/cms/cbernet/test_data/nyc_taxis/2009/yellow_tripdata_2009-11.csv'),

In [8]:
[f.stat().st_size / 1024**3 for f in t2009]

[2.3637942634522915,
 2.249560944736004,
 2.4210419980809093,
 2.406269527040422,
 2.491463811136782,
 2.394559129141271,
 2.3006204795092344,
 2.310441732406616,
 2.360710578970611,
 2.637480964884162,
 2.444788286462426,
 2.497821459546685]